In [38]:
import sys
sys.dont_write_bytecode = True
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import copy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

from process_data import *
from evaluation import *

In [39]:
# -------------------------------------------------------------------
PATH_TRAIN_LABEL_CSV_FILE = r"unlabel_data/label_train.csv"
PATH_TRAIN_UNLABEL_CSV_FILE = r"unlabel_data/unlabel_train.csv"
PATH_TEST_CSV_FILE = r"unlabel_data/test.csv"

NUM_FEATURES = 14
OUTPUT_DIMS = 8

TABNET_PRETRAIN_RATIO = 0.5

NUM_CLIENTS = 10
NUM_ROUNDS = 14

MAX_EPOCHS_LOCAL = 7
MAX_EPOCHS_SERVER = 100
# -------------------------------------------------------------------


In [40]:
# 1. Read dataset
(X_train_label, y_train_label, list_labels) = Read_Dataset(PATH_TRAIN_LABEL_CSV_FILE)
print("Y train label: {}".format(Counter(y_train_label)))

(X_train_unlabel, _) = Read_Dataset(PATH_TRAIN_UNLABEL_CSV_FILE, is_unlabel=True)

scaler = MinMaxScaler()
scaler = scaler.fit(X_train_label)
X_train_label = scaler.transform(X_train_label)
X_train_unlabel = scaler.transform(X_train_unlabel)

# 2. Split UNLABEL into clients
list_client_data = np.array_split(X_train_unlabel, NUM_CLIENTS)

Y train label: Counter({0: 6141, 2: 1525, 4: 790, 7: 637, 1: 259, 6: 210, 3: 78, 5: 61})


In [41]:
# 3. Define global model (self superviesed TabNet)
unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
)

# Init global model (self-supervised) by labeled dataset in server
unsupervised_model.fit(
            X_train = X_train_label,        
            eval_set = [X_train_label],
            max_epochs=MAX_EPOCHS_SERVER,
            pretraining_ratio=TABNET_PRETRAIN_RATIO,
            warm_start = True
)

/home/necphy/miniconda3/envs/thaivu_env/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 882.17027| val_0_unsup_loss_numpy: 64.06754302978516|  0:00:00s
epoch 1  | loss: 87.82187| val_0_unsup_loss_numpy: 12.097559928894043|  0:00:00s
epoch 2  | loss: 20.9383 | val_0_unsup_loss_numpy: 6.32997989654541|  0:00:00s
epoch 3  | loss: 31.33524| val_0_unsup_loss_numpy: 4.4753499031066895|  0:00:00s
epoch 4  | loss: 4.54167 | val_0_unsup_loss_numpy: 2.909440040588379|  0:00:00s
epoch 5  | loss: 4.48936 | val_0_unsup_loss_numpy: 1.9676200151443481|  0:00:00s
epoch 6  | loss: 5.62658 | val_0_unsup_loss_numpy: 1.5910300016403198|  0:00:01s
epoch 7  | loss: 2.93016 | val_0_unsup_loss_numpy: 1.4351999759674072|  0:00:01s
epoch 8  | loss: 2.55867 | val_0_unsup_loss_numpy: 1.2166399955749512|  0:00:01s
epoch 9  | loss: 6.51249 | val_0_unsup_loss_numpy: 1.3930699825286865|  0:00:01s
epoch 10 | loss: 21.33544| val_0_unsup_loss_numpy: 1.1799499988555908|  0:00:01s
epoch 11 | loss: 2.59376 | val_0_unsup_loss_numpy: 1.2411400079727173|  0:00:01s
epoch 12 | loss: 5.863   | val_

/home/necphy/miniconda3/envs/thaivu_env/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [42]:
# 4. Federated Learning process
list_client_models = []
for i in range(NUM_CLIENTS):
    client_model = copy.deepcopy(unsupervised_model)
    list_client_models.append(client_model)


for idx_round in range(NUM_ROUNDS):
    print("---------- [INFO] Round {}".format(idx_round))

    # Train client (local) model
    for idx_client in range(NUM_CLIENTS):
        print("----- [INFO] Client {}".format(idx_client))

        # Client get weight from global model
        list_client_models[idx_client].network.load_state_dict(unsupervised_model.network.state_dict())
        
        # Train local model in each user's data
        list_client_models[idx_client].fit(
            X_train = list_client_data[idx_client],
            eval_set = [list_client_data[idx_client]],
            max_epochs=MAX_EPOCHS_LOCAL,
            pretraining_ratio=TABNET_PRETRAIN_RATIO,
            warm_start = True  
        )


    # Compute the average of the local model weights
    unsupervised_model_state = unsupervised_model.network.state_dict()
    for name, param in unsupervised_model.network.named_parameters():

        list_client_layer = []
        for idx_client in range(NUM_CLIENTS):
            current_layer = list_client_models[idx_client].network.state_dict()[name]
            list_client_layer.append(current_layer)

        new_param = torch.mean(torch.stack(list_client_layer), dim=0)
        unsupervised_model_state[name] = new_param

    # Update global (self-supervised) model 
    unsupervised_model.network.load_state_dict(unsupervised_model_state)

---------- [INFO] Round 0
----- [INFO] Client 0
epoch 0  | loss: 7.17616 | val_0_unsup_loss_numpy: 4.171639919281006|  0:00:00s
epoch 1  | loss: 5.20478 | val_0_unsup_loss_numpy: 1.7467199563980103|  0:00:00s
epoch 2  | loss: 2.05766 | val_0_unsup_loss_numpy: 2.069890022277832|  0:00:00s
epoch 3  | loss: 2.33396 | val_0_unsup_loss_numpy: 1.2552900314331055|  0:00:00s
epoch 4  | loss: 2.18519 | val_0_unsup_loss_numpy: 1.924530029296875|  0:00:00s
epoch 5  | loss: 34.42152| val_0_unsup_loss_numpy: 3.012010097503662|  0:00:00s
epoch 6  | loss: 1.94342 | val_0_unsup_loss_numpy: 1.9385199546813965|  0:00:00s
Stop training because you reached max_epochs = 7 with best_epoch = 3 and best_val_0_unsup_loss_numpy = 1.2552900314331055
----- [INFO] Client 1


/home/necphy/miniconda3/envs/thaivu_env/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 3.39686 | val_0_unsup_loss_numpy: 7.251180171966553|  0:00:00s
epoch 1  | loss: 4.68909 | val_0_unsup_loss_numpy: 1.933019995689392|  0:00:00s
epoch 2  | loss: 2.02019 | val_0_unsup_loss_numpy: 1.2371000051498413|  0:00:00s
epoch 3  | loss: 1.43663 | val_0_unsup_loss_numpy: 1.7541500329971313|  0:00:00s
epoch 4  | loss: 1.50879 | val_0_unsup_loss_numpy: 1.3800100088119507|  0:00:00s
epoch 5  | loss: 1.33194 | val_0_unsup_loss_numpy: 1.2385799884796143|  0:00:00s
epoch 6  | loss: 1.35526 | val_0_unsup_loss_numpy: 1.8370399475097656|  0:00:00s
Stop training because you reached max_epochs = 7 with best_epoch = 2 and best_val_0_unsup_loss_numpy = 1.2371000051498413
----- [INFO] Client 2
epoch 0  | loss: 21.76027| val_0_unsup_loss_numpy: 3.968559980392456|  0:00:00s


/home/necphy/miniconda3/envs/thaivu_env/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 1  | loss: 12.55578| val_0_unsup_loss_numpy: 5.9730401039123535|  0:00:00s
epoch 2  | loss: 30.85536| val_0_unsup_loss_numpy: 3.9293200969696045|  0:00:00s
epoch 3  | loss: 6.17868 | val_0_unsup_loss_numpy: 2.402669906616211|  0:00:00s
epoch 4  | loss: 1.54867 | val_0_unsup_loss_numpy: 2.0787899494171143|  0:00:00s
epoch 5  | loss: 8.74115 | val_0_unsup_loss_numpy: 8.415769577026367|  0:00:00s
epoch 6  | loss: 2.47144 | val_0_unsup_loss_numpy: 3.263279914855957|  0:00:00s
Stop training because you reached max_epochs = 7 with best_epoch = 4 and best_val_0_unsup_loss_numpy = 2.0787899494171143
----- [INFO] Client 3


...

/home/necphy/miniconda3/envs/thaivu_env/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 3.29853 | val_0_unsup_loss_numpy: 2.0879499912261963|  0:00:00s
epoch 1  | loss: 1.12332 | val_0_unsup_loss_numpy: 1.2160600423812866|  0:00:00s
epoch 2  | loss: 1.17091 | val_0_unsup_loss_numpy: 1.2998700141906738|  0:00:00s
epoch 3  | loss: 1.10702 | val_0_unsup_loss_numpy: 0.9353299736976624|  0:00:00s
epoch 4  | loss: 1.25861 | val_0_unsup_loss_numpy: 0.9266899824142456|  0:00:00s
epoch 5  | loss: 0.95452 | val_0_unsup_loss_numpy: 0.8819500207901001|  0:00:00s
epoch 6  | loss: 1.01812 | val_0_unsup_loss_numpy: 0.8649899959564209|  0:00:00s
Stop training because you reached max_epochs = 7 with best_epoch = 6 and best_val_0_unsup_loss_numpy = 0.8649899959564209
----- [INFO] Client 9
epoch 0  | loss: 6.96496 | val_0_unsup_loss_numpy: 2.730989933013916|  0:00:00s


/home/necphy/miniconda3/envs/thaivu_env/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 1  | loss: 2.27455 | val_0_unsup_loss_numpy: 2.0663700103759766|  0:00:00s
epoch 2  | loss: 2.59581 | val_0_unsup_loss_numpy: 1.1660100221633911|  0:00:00s
epoch 3  | loss: 1.10773 | val_0_unsup_loss_numpy: 1.2247099876403809|  0:00:00s
epoch 4  | loss: 1.11779 | val_0_unsup_loss_numpy: 1.4825600385665894|  0:00:00s
epoch 5  | loss: 1.60244 | val_0_unsup_loss_numpy: 1.7709200382232666|  0:00:00s
epoch 6  | loss: 2.64606 | val_0_unsup_loss_numpy: 1.5197499990463257|  0:00:00s
Stop training because you reached max_epochs = 7 with best_epoch = 2 and best_val_0_unsup_loss_numpy = 1.1660100221633911


/home/necphy/miniconda3/envs/thaivu_env/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


## Train KNN in embedding X

In [43]:
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score

In [48]:
# Extract embedding
embedding_X = unsupervised_model.predict(X_train_label)[0]

clf = KNeighborsClassifier(n_neighbors=2)
clf.fit(embedding_X, y_train_label)

KNeighborsClassifier(n_neighbors=2)

In [49]:
(X_test, y_test, _) = Read_Dataset(PATH_TEST_CSV_FILE)
X_test = scaler.transform(X_test)
embedding_X_test = unsupervised_model.predict(X_test)[0]

y_pred = clf.predict(embedding_X_test)

f1_score_value = f1_score(y_test, y_pred, average='macro')
print("F1 score: {}".format(f1_score_value))

F1 score: 0.13227480118586157


# Train KNN

In [46]:
clf = KNeighborsClassifier(n_neighbors=2)
clf.fit(X_train_label, y_train_label)

KNeighborsClassifier(n_neighbors=2)

In [47]:
(X_test, y_test, _) = Read_Dataset(PATH_TEST_CSV_FILE)
X_test = scaler.transform(X_test)

y_pred = clf.predict(X_test)

f1_score_value = f1_score(y_test, y_pred, average='macro')
print("F1 score: {}".format(f1_score_value))

F1 score: 0.9351128654897647
